# To SQL

In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000) ## 800GB?

from calitp.tables import tbl

from calitp import query_sql, magics
import branca

import shared_utils

from siuba import *
import pandas as pd

import datetime as dt
import time
from zoneinfo import ZoneInfo

import importlib

import gcsfs
fs = gcsfs.GCSFileSystem()

from tqdm import tqdm_notebook
from tqdm.notebook import trange, tqdm

from IPython.display import display, Markdown

import utils

/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. Yo

In [2]:
analysis_date_start = '2022-05-01'
analysis_date_end = '2022-05-31'

In [3]:
#function to change to SQL
def get_pct_ran_df(itp_id, list_of_dates, gtfs_daily, rt):
    
    pcts = []
    
    # loop through list of dates
    for single_date in list_of_dates:
        
        #filter for single day
        
        gtfs_daily2 = (gtfs_daily>>filter(_.service_date == single_date))
        rt2 = (rt>>filter(_.date == single_date))
        
        #outer join schedule and rt data 
        sched_rt_df = (pd.merge(gtfs_daily2, rt2, how='outer', on='trip_id', indicator='have_rt'))

        
        day_pct_ran = {}
        day_pct_ran['date'] = single_date
        if ((len(sched_rt_df))!=0):
            day_pct_ran['pct_trips_ran'] = ((len(sched_rt_df>>filter(_.have_rt=='both')))/(len(gtfs_daily2)))
        elif ((len(sched_rt_df))==0):
            day_pct_ran['pct_trips_ran'] = ''
        pct_ran = pd.DataFrame([day_pct_ran])
        
        # add columns with counts 
        pct_ran['n_have_rt'] = (len(sched_rt_df>>filter(_.have_rt=='both')))
        pct_ran['n_missing_rt'] = (len(sched_rt_df>>filter(_.have_rt=='right_only')))
        pct_ran['unmatched_rt'] = (len(sched_rt_df>>filter(_.have_rt=='left_only')))
        
        # add columns for number of unique trip_ids
        pct_ran['nunique_sched'] = (gtfs_daily2.trip_id.nunique())
        pct_ran['nunique_rt'] = (rt2.trip_id.nunique())

        pcts.append(pct_ran)                                                    
        #code help from: https://stackoverflow.com/questions/28669482/appending-pandas-dataframes-generated-in-a-for-loop
   
    #add each date together 
    pcts = pd.concat(pcts)
    
    #arrange by date
    pcts = pcts>>arrange(_.date)
    return pd.DataFrame(pcts)


In [ ]:
def load_schedule_data(start_date, end_date, itp_id):
    gtfs_daily = (
        tbl.views.gtfs_schedule_fact_daily_trips()
        >> filter(_.calitp_itp_id == itp_id)
        >> filter((_.service_date >= start_date),
                 (_.service_date <= end_date))
        >> filter(_.is_in_service == True)
        >> collect()
    )
    return gtfs_daily


In [7]:
%%sql
WITH 
rt_trips AS(
SELECT * 
FROM `cal-itp-data-infra-staging.natalie_views.test_rt_trips`
WHERE date BETWEEN '2022-05-01' AND '2022-05-31'
),
sched_trips AS(
SELECT *
FROM `cal-itp-data-infra.views.gtfs_schedule_fact_daily_trips`
WHERE (calitp_itp_id=300
    AND service_date >= '2022-05-01'
    AND service_date <= '2022-05-31'
    AND is_in_service = True))

SELECT *
FROM sched_trips

/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/


,feed_key,trip_key,trip_id,route_id,calitp_itp_id,calitp_url_number,service_id,service_date,service_indicator,service_start_date,...,service_inclusion,service_exclusion,is_in_service,calitp_extracted_at,calitp_deleted_at,n_stops,n_stop_times,trip_first_departure_ts,trip_last_arrival_ts,service_hours
0,4100255323842349957,6242274318901953391,882298,3487,300,0,12010,2022-05-10,0,2022-03-27,...,True,None,True,2022-03-29,2022-07-18,12,12,51240,53340,0.583333
1,4100255323842349957,190829107003865392,883654,3501,300,0,77710,2022-05-31,0,2022-03-27,...,True,None,True,2022-03-29,2022-07-18,14,14,57720,58920,0.333333
2,4100255323842349957,-167558247602636188,883620,3501,300,0,77710,2022-05-26,0,2022-03-27,...,True,None,True,2022-03-29,2022-07-18,14,14,31200,32460,0.350000
3,4100255323842349957,-7570616633128089847,883630,3501,300,0,77710,2022-05-27,0,2022-03-27,...,True,None,True,2022-03-29,2022-07-18,14,14,39000,39960,0.266667
4,4100255323842349957,-6185002435720068782,882543,3489,300,0,10,2022-05-04,1,2022-03-27,...,None,None,True,2022-03-29,2022-07-18,14,14,71220,72300,0.300000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39725,4100255323842349957,6493733988443697067,881906,3485,300,0,10,2022-05-13,1,2022-03-27,...,None,None,True,2022-03-29,2022-07-18,54,54,51600,55440,1.066667
39726,4100255323842349957,8685837590523691527,881826,3485,300,0,10,2022-05-12,1,2022-03-27,...,None,None,True,2022-03-29,2022-07-18,56,56,54600,59614,1.392778
39727,4100255323842349957,7150541492777559695,881961,3485,300,0,10,2022-05-03,1,2022-03-27,...,None,None,True,2022-03-29,2022-07-18,57,57,24000,27360,0.933333
39728,4100255323842349957,7404775785081750034,881928,3485,300,0,10,2022-05-12,1,2022-03-27,...,None,None,True,2022-03-29,2022-07-18,57,57,28500,32640,1.150000
